## PDF Reading

In [30]:
pip install pyPDF2

In [0]:
import PyPDF2

In [0]:
#Taking file input
#number of files
nf=2

file= open('Apr-18.PDF','rb')
file1= open('Apr-17.PDF','rb')


In [0]:
#A list of files
file_list=[file,file1]

In [0]:
# Reading number of pages

for i in range(nf):
  pdfReader = PyPDF2.PdfFileReader(file_list[i])
  n1=pdfReader.numPages
  pages.append(n1)

In [0]:
# A list containing all files, All_Docs[i] is i-th file 
# All_Docs[i][j] is j-th page of i-th file
All_Docs=[] 

In [0]:

for j in range(n):
  pdfReader = PyPDF2.PdfFileReader(file_list[j])
  file_text=[] #takes string of page text
  for i in range(pages[j]):
    pageObj = pdfReader.getPage(i)
    a=pageObj.extractText()
    file_text.append(a)
  
  All_Docs.append(file_text)

In [38]:
type(All_Docs[0][1]),type(All_Docs[0])

(str, list)

# Finding Optimal number of Clusters

In [0]:
Dup_All_Docs=All_Docs[:]

In [40]:
type(Dup_All_Docs[0][0])

str

In [0]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score

import pandas as pd
from nltk.tokenize import RegexpTokenizer
#from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
from nltk.corpus import stopwords
import gensim

from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np

In [42]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
tokenizer = RegexpTokenizer(r'\w+')
en_stop = set(stopwords.words('english'))
en_stop.add('the')#The is not in stopwords

In [0]:
tokens=[]


for j in Dup_All_Docs:
  x=[]
  for i in j:
    token=tokenizer.tokenize(i)
    token=[i for i in token if(not(str(i).isdigit() or not(str(i).isalpha())) and len(str(i)) > 2 )]
    token=[i for i in token if( i not in en_stop)]
    x.extend(token)
  
  tokens.extend(x)

In [46]:
len(tokens),tokens[:10]

(7941,
 ['PRESS',
  'RELEASE',
  'DEPARTMENT',
  'COMMUNICATION',
  'Central',
  'Office',
  'Marg',
  'Mumbai',
  'Phone',
  'Fax'])

In [0]:
#df9['Cleaned_PaperText'] = pd.Series(tokens)
        
vectorizer = TfidfVectorizer()

Y = vectorizer.fit_transform(tokens)

In [0]:
#Not Working

distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(Y)
    kmeanModel.fit(Y)
   # distortions.append(sum(np.min(cdist(Y, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

In [77]:
#2nd Method using Silhouette Score

n=0
ss=0
for n_cluster in range(2, 11):
    kmeans = KMeans(n_clusters=n_cluster).fit(Y)
    label = kmeans.labels_
    sil_coeff = silhouette_score(X, label, metric='euclidean')  
    print("For n_clusters={}, The Silhouette Coefficient is {}".format(n_cluster, sil_coeff))
    if sil_coeff>ss:
      ss=sil_coeff
      n=n_cluster

For n_clusters=2, The Silhouette Coefficient is 0.013113115977550036
For n_clusters=3, The Silhouette Coefficient is 0.014499944484682307
For n_clusters=4, The Silhouette Coefficient is 0.030536189095659732
For n_clusters=5, The Silhouette Coefficient is 0.046154072366921935
For n_clusters=6, The Silhouette Coefficient is 0.05376507395013614
For n_clusters=7, The Silhouette Coefficient is 0.054511231206287536
For n_clusters=8, The Silhouette Coefficient is 0.059205379653494095
For n_clusters=9, The Silhouette Coefficient is 0.0714241604401495
For n_clusters=10, The Silhouette Coefficient is 0.07818054359294499


In [78]:
print("Optimal numbers of clusters are : {}".format(n))

Optimal numbers of clusters are : 10


# LDA

In [0]:

import nltk
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.wrappers import LdaMallet

In [51]:
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:

dictionary=Dictionary([[i] for i in tokens])

In [53]:
print(dictionary)

Dictionary(2246 unique tokens: ['PRESS', 'RELEASE', 'DEPARTMENT', 'COMMUNICATION', 'Central']...)


In [80]:
X=[dictionary.doc2bow([sents]) for sents in tokens]#getting bag of words.
tfidf=TfidfModel(X)

print(type(tfidf))
X=tfidf[X]


<class 'gensim.models.tfidfmodel.TfidfModel'>


In [0]:
from gensim.models import LdaMulticore,CoherenceModel

lda=LdaMulticore(X,num_topics=n,id2word=dictionary,workers=8)
#lda.print_topics()

In [0]:

ldatopics = [[(word,prob) for word, prob in topic] for topicid, topic in lda.show_topics(formatted=False)]
lda.show_topics(formatted=True)

In [0]:
index=["Most Common Word"]
col=[]
a='Topic-'
c=''
for i in range(n):
  c+=a+str(i+1)
  col.append(c)
  c=''

topic_df=pd.DataFrame(data=ldatopics,columns=col)

In [147]:
topic_df

,Topic-1,Topic-2,Topic-3,Topic-4,Topic-5,Topic-6,Topic-7,Topic-8,Topic-9,Topic-10
0,February,expected,impact,second,remained,March,deficit,Bank,past,indicators
1,growth,March,inflation,headline,target,although,cent,Bank,goods,months
2,per,cent,food,may,January,level,high,imports,trade,India
3,services,global,The,outlook,output,domestic,inflation,year,economic,data
4,Reserve,year,risks,activity,The,months,macroeconomic,monetary,February,pick
5,prices,per,growth,rate,stance,risks,Policy,upside,consumption,low
6,The,MPC,rates,effects,projected,Bank,term,well,given,repo
7,inflation,cent,food,rate,demand,factors,impact,prices,policy,one
8,policy,CPI,investment,resolution,effects,January,growth,crude,markets,since
9,also,production,Committee,meeting,likely,price,rising,month,ahead,manufacturing


# Evaluating Model


In [99]:
tk=[[i] for i in tokens]
ldatopics = [[word for word, prob in topic] for topicid, topic in lda.show_topics(formatted=False)]
lda_coherence = CoherenceModel(topics=ldatopics, texts=tk, dictionary=dictionary, window_size=10).get_coherence()
lda_coherence

0.7116011974877247

In [0]:
coherence_score=[]
tk=[[i] for i in tokens]
for i in range(n):
  lda=LdaMulticore(X,num_topics=n,id2word=dictionary,workers=8)
  ldatopics = [[word for word, prob in topic] for topicid, topic in lda.show_topics(formatted=False)]
  lda_coherence = CoherenceModel(topics=ldatopics, texts=tk, dictionary=dictionary, window_size=10).get_coherence()
  coherence_score.append(lda_coherence)



In [101]:
print("Max Score is: {} for {} clusters".format(max(coherence_score),coherence_score[coherence_score.index(max(coherence_score))]))

[0.7171352722194907,
 0.71508921022708,
 0.7172028234261173,
 0.7170485826526457,
 0.7150678370468146,
 0.7130522665121717,
 0.7143456995668166,
 0.7137254293558761,
 0.7154616691118152,
 0.7144941822309956]

In [103]:
print("Max Score is: {} for {} clusters".format(max(coherence_score),coherence_score.index(max(coherence_score))))

Max Score is: 0.7172028234261173 for 2 clusters


# Misc.

In [0]:
Dup_All_Docs=All_Docs[0]+All_Docs[1]
df=[]

In [0]:
for j in All_Docs:
  for i in j:
    df.append(i)


In [122]:
len(All_Docs[0])

14

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vectoriser = CountVectorizer().fit(Dup_All_Docs)
X = vectoriser.transform(Dup_All_Docs)
df=pd.DataFrame(X.A, columns=sorted(vectoriser.vocabulary_.keys(), key=lambda k: vectoriser.vocabulary_[k]))

In [133]:
df_page=pd.DataFrame(data=Dup_All_Docs)
df_page.head()

,0
0,"1 \n \n PRESS\n RELEASE\n \n \n, \n \n, \n\..."
1,2 Resolution\n 5. On \nthe\n basis of an asse...
2,"3 month low in\n March, have recently witness..."
3,"4 acreage and low soil moisture, but imports ..."
4,5 large tax \ncollections. After turning into...
